In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

In [ ]:
# CONSTANTS
DELAY = 3
SEASON_RANGE = list(range(2022, 2020, -1))
COMP = "Premier-League" # IMPORTANTE (A URL ABAIXO ESTÁ DINAMICAMENTE PEGANDO ESSA CONSTANTE)
URL = f"https://fbref.com/en/comps/9/{COMP}-Stats" # URL constante (usada como base) da premier league, alterar para o Brasileirão, se necessário

In [ ]:
url = URL # url não constante, pois será alterada ao longo do fluxo

**VAMOS CAPTURAR AS TABELAS GERAIS DE JOGADORES E TIMES** 

*1 - CAPTURANDO A TABELA DE INFORMAÇÕES GERAIS DOS JOGADORES (Standard Stats)*

In [ ]:
teams_players_data_table = []

for year in SEASON_RANGE:
    data = requests.get(url) # Faz um request em toda a url
    soup = BeautifulSoup(data.text) # Retorna o texto html do retorno
    table = soup.select('table.stats_table')[0] # Coletamos a primeira tabela
 
    links = table.find_all('a') # Capturamos todas as ancoras presentes nessa tabela
    links = [l.get("href") for l in links] # Capturamos somente os links
    links = [l for l in links if '/squads/' in l] # Pegamos somente os links que possui "/SQUADS/" no meio, pois como capturamos todas as ancoras na tabela, tem outros links irrelevantes presentes
    team_urls = [f"https://fbref.com{l}" for l in links] # Criamos uma lista com os links já estruturados para cada time
    
    previous_season = soup.select("#meta > div:nth-child(2) > div")[0].find('a').get("href") # Capturamos o link da season anterior
    url = f"https://fbref.com{previous_season}" # Capturamos o link da season anterior
    
    # Obtém os dados da página de cada time
    for team_url in team_urls:
        response = requests.get(team_url)

        try:
            # Extrai os dados da tabela "Standard Stats" em forma de dataframe
            team_player_data = pd.read_html(response.text, match="Standard Stats")[0]
            team_player_data.columns = team_player_data.columns.droplevel() # Dropamos a primeira coluna
        
            team_name = team_url.split("/")[-1].replace("-Stats", "").replace("-", " ") # Pegamos o nome do time através da url
            
            # Criações de features (Colunas)
            team_player_data["Season"] = year
            team_player_data["Team"] = team_name
            team_player_data["Comp"] = COMP

            # Adiciona os dados à lista de resultados
            teams_players_data_table.append(team_player_data)

        except Exception as e:
            print(f"""A solicitação para {team_url} falhou.
                  Erro: {e} \n
                  código de status: {response.status_code}""")
            
        time.sleep(DELAY)

*2 - CAPTURANDO A TABELA DE INFORMAÇÃO GERAL DOS TIMES (Scores and Fixtures)*

In [ ]:
teams_data_table = []

for year in SEASON_RANGE:
    data = requests.get(url)
    soup = BeautifulSoup(data.text)
    table = soup.select('table.stats_table')[0]

    links = [l.get("href") for l in table.find_all('a')]
    links = [l for l in links if '/squads/' in l]
    team_urls = [f"https://fbref.com{l}" for l in links]
    
    previous_season = soup.select("#meta > div:nth-child(2) > div")[0].find('a').get("href")
    url = f"https://fbref.com{previous_season}"
    
    # Obtém os dados da página de cada time
    for team_url in team_urls:
        response = requests.get(team_url)

        # Verifica se a solicitação foi bem-sucedida
        try:
            # Extrai os dados da segunda tabela "Scores and Fixtures" em forma de dataframe
            team_data = pd.read_html(response.text, match="Scores & Fixtures")[0]
        
            team_name = team_url.split("/")[-1].replace("-Stats", "").replace("-", " ") # Pegamos o nome do time através da url

            # Criações de features (colunas)
            team_data["Season"] = year
            team_data["Team"] = team_name
            team_data["Comp"] = COMP

            # Adiciona os dados à lista "teams_data_table"
            teams_data_table.append(team_data)

        except Exception as e:
            print(f"""A solicitação para {team_url} falhou.
                  Erro: {e} \n
                  código de status: {response.status_code}""")
            
        time.sleep(DELAY)

***PARA VISUALIZAR AS TABELAS***

In [ ]:
teams_players_data_table[0].head() #Visualização do primeiro elemento

In [ ]:
teams_data_table[0].head() #Visualização do primeiro elemento